In [ ]:
import heapq

class Puzzle:
    goal_state = [
        [1, 2, 3],
        [4, 5, 6],
        [7, 8, 0]
    ]
    heuristic = None
    evaluation_function = None
    needs_hueristic = False
    num_of_instances = 0

    def __init__(self, state, parent, action, path_cost, needs_hueristic=False):
        self.parent = parent
        self.state = state
        self.action = action
        if parent:
            self.path_cost = parent.path_cost + path_cost
        else:
            self.path_cost = path_cost
        if needs_hueristic:
            self.needs_hueristic = True
            self.calculate_heuristic()
            self.evaluation_function = self.heuristic + self.path_cost
        Puzzle.num_of_instances += 1

    def __str__(self):
        return str(self.state[0]) + "\n" + str(self.state[1]) + "\n" + str(self.state[2]) + "\n"

    def __lt__(self, other):
        return self.evaluation_function < other.evaluation_function

    def is_goal(self):
        return self.state == self.goal_state

    def calculate_heuristic(self):
        self.heuristic = 0
        for row in range(3):
            for col in range(3):
                number = self.state[row][col]
                if number != 0:
                    goal_row, goal_col = divmod(number-1, 3) # subtracting 1 for proper indexing
                    self.heuristic += abs(goal_row - row) + abs(goal_col - col)

    def possible_moves(self):
        row, col = None, None
        for i in range(3):
            for j in range(3):
                if self.state[i][j] == 0:
                    row, col = i, j
                    break

        possible_actions = ['U', 'D', 'L', 'R']
        if row == 0:
            possible_actions.remove('U')
        if row == 2:
            possible_actions.remove('D')
        if col == 0:
            possible_actions.remove('L')
        if col == 2:
            possible_actions.remove('R')

        return possible_actions

    def generate_child(self, action):
        row, col = None, None
        for i in range(3):
            for j in range(3):
                if self.state[i][j] == 0:
                    row, col = i, j
                    break

        new_state = [x[:] for x in self.state]
        if action == 'U':
            new_state[row][col], new_state[row-1][col] = new_state[row-1][col], new_state[row][col]
        elif action == 'D':
            new_state[row][col], new_state[row+1][col] = new_state[row+1][col], new_state[row][col]
        elif action == 'L':
            new_state[row][col], new_state[row][col-1] = new_state[row][col-1], new_state[row][col]
        elif action == 'R':
            new_state[row][col], new_state[row][col+1] = new_state[row][col+1], new_state[row][col]

        return Puzzle(new_state, self, action, 1, self.needs_hueristic)

def a_star_search(initial_state):
    count = 0
    explored = set()
    start_node = Puzzle(initial_state, None, None, 0, True)
    q = [start_node]

    while q:
        node = heapq.heappop(q)

        if str(node.state) in explored:
            continue

        explored.add(str(node.state))

        print(f"Move: {node.action if node.action else 'Start'}")
        print(f"g(n): {node.path_cost}")
        print(f"h(n): {node.heuristic}")
        print(node)
        print("-------")

        if node.is_goal():
            return node, count

        children = [node.generate_child(action) for action in node.possible_moves()]
        for child in children:
            if str(child.state) not in explored:
                count += 1
                heapq.heappush(q, child)

    return None

if __name__ == "__main__":
    initial_state = [
        [2, 3, 0],
        [1, 5, 6],
        [4, 7, 8]
    ]

    result, count = a_star_search(initial_state)
    moves = []
    while result.parent:
        moves.insert(0, result.action)
        result = result.parent
    print("Moves:", moves)
    print("Total Moves:", len(moves))

Move: Start
g(n): 0
h(n): 6
[2, 3, 0]
[1, 5, 6]
[4, 7, 8]

-------
Move: L
g(n): 1
h(n): 5
[2, 0, 3]
[1, 5, 6]
[4, 7, 8]

-------
Move: L
g(n): 2
h(n): 4
[0, 2, 3]
[1, 5, 6]
[4, 7, 8]

-------
Move: D
g(n): 3
h(n): 3
[1, 2, 3]
[0, 5, 6]
[4, 7, 8]

-------
Move: D
g(n): 4
h(n): 2
[1, 2, 3]
[4, 5, 6]
[0, 7, 8]

-------
Move: R
g(n): 5
h(n): 1
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

-------
Move: R
g(n): 6
h(n): 0
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

-------
Moves: ['L', 'L', 'D', 'D', 'R', 'R']
Total Moves: 6


In [ ]:
pip install dtreeviz

# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import dtreeviz

# Load the Breast Cancer dataset
data = load_breast_cancer()
X = data.data
y = data.target

species_names = ['benign', 'malignant']
y_labels = [species_names[label] for label in y]
print("_______________")

# Create a DataFrame from the dataset
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = y_labels  # Add the species labels as a new column

# Display specific fields for 'malignant' (class 0)
malignant_data = df[df['target'] == 'malignant']
print("Data for malignant cancer (Class 0):")
print(malignant_data.head(3))
print("_______________")
# Display specific fields for 'benign' (class 1)
benign_data = df[df['target'] == 'benign']
print("\nData for 'benign cancer' (Class 1):")
print(benign_data.head(3))

# Prepare Data for Binary Classification: Classify between 'benign' (class 1) and 'malignant' (class 0)
y_binary = (y == 1).astype(int)  # 1 for 'benign' and 0 for 'malignant'


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.3, random_state=42)


# Create a decision tree classifier
clf = DecisionTreeClassifier()
# Train (fit) the classifier on the training data
clf.fit(X_train, y_train)
# Make predictions on the test data
y_pred = clf.predict(X_test)
# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


print("_______________")
# Visualize the decision tree using 'dtreeviz' MALIGNANT and BENIGN
m = dtreeviz.model(clf, X_train, y_train, target_name='Cancer type', feature_names=data.feature_names, class_names=['malignant', 'benign'])
m.view()



from sklearn.tree import DecisionTreeClassifier, plot_tree

import matplotlib.pyplot as plt


# Visualize the decision tree using Matplotlib
plt.figure(figsize=(30, 10))
plot_tree(clf, filled=True, feature_names=data.feature_names, class_names=['malignant', 'benign'], fontsize=10)
plt.show()